# PyLops V2, what's new?

**Author: M. Ravasi, KAUST**

The aim of this second tutorial is to introduce you to our latest creation (still in the making): **PyLops V2**

As any new Major version, there will be breaking changes! 

Nevetheless, this is the very first time since the initial creation of PyLops that we decided to sit back and revisit some the early design choices. We hope the benefit introduced by such changes outweights the pain of making a few changes to your codes.

In the following we want to give you a taste of some new features and highlight along the way if (and how) these have required breaking changes.

For more details on how to quickly migrate your PyLops v1.x codes into PyLops v2.x codes please refer to this [document](https://github.com/PyLops/pylops/blob/dev/MIGRATION_V1_V2.md).

## Useful links

- Tutorial Github repository: https://github.com/PyLops/pylops_transform2022
        
- PyLops Github repository: https://github.com/PyLops/pylops

- PyLops reference documentation: https://pylops.readthedocs.io/en/latest/